# Network Deliberation ABM
## Discrete state, interpreted NK Model task

In [ ]:
# Imports
import configparser
import math
import random

import networkx as nx
import nkmodel as nk
import numpy as np
from numpy import random as nprand
import repsci
from tqdm.notebook import tqdm

import netdelib.soclearn.evaluate as sleval
import netdelib.soclearn.models.interpreted as slint
import netdelib.soclearn.strategy as slstrat
from netdelib.topologies.factories import *

from discrete_interpreted_nk import run_nk_trial

In [ ]:
# Configuration
config = configparser.ConfigParser()
config.read('discrete_interpreted_nk.cfg')

N = config.getint('abm', 'N')
M = config.getint('abm', 'M')
runs = config.getint('abm', 'runs')
stages = config.getint('abm', 'stages')
steps = config.getint('abm', 'steps')

bit_count = config.getint('abm', 'bit_count')
K = config.getint('abm', 'K')

small_world_k = config.getint('networks', 'small_world_k')
small_world_a = config.getfloat('networks', 'small_world_a')
barabasi_albert_m = config.getint('networks', 'barabasi_albert_m')

# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update({
    'figure.figsize': (20, 20),
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False})

networks = {
    'Complete': CompleteFactory(N, M),
    'Pref. Attach.': PreferentialFactory(N, M, barabasi_albert_m),
    'Small World': SmallWorldFactory(N, M, small_world_k, small_world_a),
    'Long Path': LongPathFactory(N, M),
    'Random Group': RandomGroupFactory(N, M)   
}

learning_strategies = {
    'Best Neighbor': slstrat.best_neighbor,
#    'Random bit': random_neighbor_bit,
#    'Random list': random_neighbor_list,
    'Conform': slstrat.conform,
    'Local Majority': slstrat.local_majority,
}

# Run Simulation 

In [ ]:
experiment = repsci.Experiment("discrete_interpreted_nk", config=config)
log = experiment.get_logger()

num_net = len(networks)
num_strat = len(learning_strategies)
score = [[0 for strat in range(num_net)] for net in range(num_strat)]

for i, (title, learning_strategy) in enumerate(tqdm(learning_strategies.items())):
    log.info('Running trials for strategy: {}'.format(title))
    
    for j, (title2, factory) in enumerate(networks.items()):
        log.info('....Running trials for network: {}'.format(title2))
        
        # Record time series for scores
        run_score = []
        for run in range(runs):
            model = nk.NK(bit_count, K)
            beliefs_at_step = run_nk_trial(
                factory,
                learning_strategy,
                model,
                N, M, stages, steps)
            scores_for_step_agent = [model.get_values(beliefs) for beliefs in beliefs_at_step]
            # Have to convert scores from dict values to list, otherwise np gets confused
            score_for_step = [
                np.mean(list(scores.values()))
                for scores in scores_for_step_agent]
            run_score.append(score_for_step)
        # Mean/std score for all runs
        mean = np.mean(run_score, axis=0)
        std = np.std(run_score, axis=0)
        score[i][j] = (mean, std)
        
log.info('All trials complete')

In [ ]:
plt.figure(figsize=(15, 9))
for i, (title, learning_strategy) in enumerate(learning_strategies.items()):
    for j, (title2, factory) in enumerate(networks.items()):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)

        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b:')

        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Correct')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.tight_layout()
        plt.ylim([0, 1])

out = experiment.get_filename('plot.png')
plt.savefig(out, dpi=300)